In [31]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = {}
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols[:5]:
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    # Step 2.ii: Fetch data from yfinance
    end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
    start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
    stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

    # Step 2.iii: Calculate % return for each day
    stock_data['Prev Close'] = stock_data['Close'].shift(1)
    stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

    # Step 2.iv: Check for gain or loss and update short_listed_stocks
    if stock_data['% Return'].iloc[-1] > threshold_positive:
        short_listed_stocks[symbol] = 1
    elif stock_data['% Return'].iloc[-1] < threshold_negative:
        short_listed_stocks[symbol] = -1

    # Step 2.vi: Calculate pivots
    prev_high = stock_data['High'].iloc[-2]
    prev_low = stock_data['Low'].iloc[-2]
    prev_close = stock_data['Close'].iloc[-2]
    pivot = (prev_high + prev_low + prev_close) / 3
    res1 = 2 * pivot - prev_low
    res2 = pivot + (res1 - (2 * pivot - prev_high))
    res3 = prev_high + 2 * (pivot - prev_low)
    sup1 = 2 * pivot - prev_high
    sup2 = pivot - (res1 - sup1)
    sup3 = prev_low - 2 * (prev_high - pivot)

    # Step 2.vii: Append pivot points to short_listed_stocks
    if symbol in short_listed_stocks:
        short_listed_stocks[symbol] = {}  # Initialize the dictionary for the symbol
        short_listed_stocks[symbol]['PIVOT'] = pivot
        short_listed_stocks[symbol]['RES1'] = res1
        short_listed_stocks[symbol]['RES2'] = res2
        short_listed_stocks[symbol]['RES3'] = res3
        short_listed_stocks[symbol]['SUP1'] = sup1
        short_listed_stocks[symbol]['SUP2'] = sup2
        short_listed_stocks[symbol]['SUP3'] = sup3
    time.sleep(0.5)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [33]:
pd.DataFrame(short_listed_stocks)

,ABB,ABBOTINDIA,ADANIENT
PIVOT,4007.149984,21680.783203,2512.483317
RES1,4077.499919,21811.566406,2539.666585
RES2,4210.349935,22020.482422,2587.183268
RES3,4280.699870,22151.265625,2614.366536
SUP1,3874.299967,21471.867188,2464.966634
SUP2,3803.950033,21341.083984,2437.783366
SUP3,3671.100016,21132.167969,2390.266683


In [18]:
start_date

datetime.date(2023, 5, 27)

In [19]:
end_date

datetime.date(2023, 6, 3)

In [20]:
stock_data

,Open,High,Low,Close,Adj Close,Volume,Prev Close,% Return
Date,,,,,,,,
2023-05-29,2570.000000,2583.500000,2523.000000,2534.050049,2534.050049,2779432,NaN,NaN
2023-05-30,2529.300049,2554.149902,2475.149902,2496.449951,2496.449951,1736953,2534.050049,-1.483795
2023-05-31,2482.000000,2573.000000,2428.000000,2493.350098,2493.350098,4268439,2496.449951,-0.124170
2023-06-01,2499.899902,2560.000000,2485.300049,2492.149902,2492.149902,1704946,2493.350098,-0.048136
2023-06-02,2535.000000,2538.899902,2425.000000,2445.050049,2445.050049,5543791,2492.149902,-1.889929


In [21]:
stock_data['% Return'].iloc[-1]

-1.8899285902236378

In [42]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.ERROR,
                    format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger()
logger.showLineNumber = True

# Disable logs from yfinance module
logging.getLogger("yfinance").setLevel(logging.WARNING)

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = []
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols:
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    try:
        # Step 2.ii: Fetch data from yfinance
        end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
        start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
        stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

        # Check if data is available for the stock
        if stock_data.empty:
            logging.warning(f"No data available for stock: {symbol}")
            continue

        # Step 2.iii: Calculate % return for each day
        stock_data['Prev Close'] = stock_data['Close'].shift(1)
        stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

        # Step 2.iv: Check for gain or loss and update short_listed_stocks
        if stock_data['% Return'].iloc[-1] > threshold_positive:
            short_listed_stocks.append({'symbol': symbol, 'flag': 1, 'percent_return': stock_data['% Return'].iloc[-1]})
        elif stock_data['% Return'].iloc[-1] < threshold_negative:
            short_listed_stocks.append({'symbol': symbol, 'flag': -1, 'percent_return': stock_data['% Return'].iloc[-1]})

        # Step 2.vi: Calculate pivots
        prev_high = stock_data['High'].iloc[-2]
        prev_low = stock_data['Low'].iloc[-2]
        prev_close = stock_data['Close'].iloc[-2]
        pivot = (prev_high + prev_low + prev_close) / 3
        res1 = 2 * pivot - prev_low
        res2 = pivot + (res1 - (2 * pivot - prev_high))
        res3 = prev_high + 2 * (pivot - prev_low)
        sup1 = 2 * pivot - prev_high
        sup2 = pivot - (res1 - sup1)
        sup3 = prev_low - 2 * (prev_high - pivot)

        # Step 2.vii: Append pivot points to short_listed_stocks
        
        short_listed_stocks[-1]['PIVOT'] = pivot
        short_listed_stocks[-1]['RES1'] = res1
        short_listed_stocks[-1]['RES2'] = res2
        short_listed_stocks[-1]['RES3'] = res3
        short_listed_stocks[-1]['SUP1'] = sup1
        short_listed_stocks[-1]['SUP2'] = sup2
        short_listed_stocks[-1]['SUP3'] = sup3

        time.sleep(0.5)
    
    except Exception as e:
        logging.error(f"Error occurred for stock: {symbol}. Error message: {str(e)}")

[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:26,395 - ERROR - Error occurred for stock: ABB. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:26,786 - ERROR - Error occurred for stock: ACC. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

...

2023-06-04 17:36:27,545 - ERROR - Error occurred for stock: ABBOTINDIA. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:27,943 - ERROR - Error occurred for stock: ADANIENT. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:28,291 - ERROR - Error occurred for stock: ADANIGREEN. Error message: list index out of range


In [38]:
pd.DataFrame(short_listed_stocks)

,symbol,flag,percent_return,PIVOT,RES1,RES2,RES3,SUP1,SUP2,SUP3
0,ABB,1,1.949480,772.400004,779.500020,789.250041,796.350057,762.649984,755.549967,745.799947
1,ABBOTINDIA,1,1.502592,21680.783203,21811.566406,22020.482422,22151.265625,21471.867188,21341.083984,21132.167969
2,ADANIENT,-1,-1.889929,2512.483317,2539.666585,2587.183268,2614.366536,2464.966634,2437.783366,2390.266683


In [48]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.ERROR,
                    format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger()
logger.showLineNumber = True

# Disable logs from yfinance module
logging.getLogger("yfinance").setLevel(logging.WARNING)

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = []
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols:
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    # Step 2.ii: Fetch data from yfinance
    end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
    start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
    try:
        stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

        # Check if data is available for the stock
        if stock_data.empty:
            logging.warning(f"No data available for stock: {symbol}")
            continue

        # Step 2.iii: Calculate % return for each day
        stock_data['Prev Close'] = stock_data['Close'].shift(1)
        stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

        # Step 2.iv: Check for gain or loss and update short_listed_stocks
        if stock_data['% Return'].iloc[-1] > threshold_positive:
            short_listed_stocks.append({'symbol': symbol, 'flag': 1, 'percent_return': stock_data['% Return'].iloc[-1]})
        elif stock_data['% Return'].iloc[-1] < threshold_negative:
            short_listed_stocks.append({'symbol': symbol, 'flag': -1, 'percent_return': stock_data['% Return'].iloc[-1]})

        # Step 2.vi: Calculate pivots
        prev_high = stock_data['High'].iloc[-2]
        prev_low = stock_data['Low'].iloc[-2]
        prev_close = stock_data['Close'].iloc[-2]
        pivot = (prev_high + prev_low + prev_close) / 3
        res1 = 2 * pivot - prev_low
        res2 = pivot + (res1 - (2 * pivot - prev_high))
        res3 = prev_high + 2 * (pivot - prev_low)
        sup1 = 2 * pivot - prev_high
        sup2 = pivot - (res1 - sup1)
        sup3 = prev_low - 2 * (prev_high - pivot)

        short_listed_stocks[symbol]['PIVOT'] = pivot
        short_listed_stocks[symbol]['RES1'] = res1
        short_listed_stocks[symbol]['RES2'] = res2
        short_listed_stocks[symbol]['RES3'] = res3
        short_listed_stocks[symbol]['SUP1'] = sup1
        short_listed_stocks[symbol]['SUP2'] = sup2
        short_listed_stocks[symbol]['SUP3'] = sup3

        stock_dict = {'symbol': symbol, 'PIVOT': pivot, 'RES1': res1, 'RES2': res2, 'RES3': res3,
                      'SUP1': sup1, 'SUP2': sup2, 'SUP3': sup3}
        short_listed_stocks.append(stock_dict)

        time.sleep(0.5)
    except Exception as e:
        logging.error(f"Error occurred for stock: {symbol}. Error message: {str(e)}")

[*********************100%***********************]  1 of 1 completed

2023-06-04 17:55:26,129 - ERROR - Error occurred for stock: NYKAA. Error message: list indices must be integers or slices, not str



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:55:26,531 - ERROR - Error occurred for stock: ZOMATO. Error message: list indices must be integers or slices, not str



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:55:26,938 - ERROR - Error occurred for stock: ZYDUSLIFE. Error message: list indices must be integers or slices, not str


In [47]:
short_listed_stocks

[{'symbol': 'NYKAA', 'flag': 1, 'percent_return': 7.609986807770045},
 {'symbol': 'ZOMATO', 'flag': 1, 'percent_return': 4.70935205497386}]

In [50]:
stock_data['High'].iloc[-2]

514.4000244140625

In [51]:
short_listed_stocks

[{'symbol': 'NYKAA', 'flag': 1, 'percent_return': 7.609986807770045},
 {'symbol': 'ZOMATO', 'flag': 1, 'percent_return': 4.70935205497386}]

In [64]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.ERROR,
                    format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger()
logger.showLineNumber = True

# Disable logs from yfinance module
logging.getLogger("yfinance").setLevel(logging.WARNING)

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = []
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols:
    stock_dict ={}
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    # Step 2.ii: Fetch data from yfinance
    end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
    start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
    stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

    # Check if data is available for the stock
    if stock_data.empty:
        logging.warning(f"No data available for stock: {symbol}")
        continue

    # Step 2.iii: Calculate % return for each day
    stock_data['Prev Close'] = stock_data['Close'].shift(1)
    stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

    # Step 2.iv: Check for gain or loss and update short_listed_stocks
    stock_dict['symbol'] = symbol
    stock_dict['percent_return'] = stock_data['% Return'].iloc[-1]
    if stock_data['% Return'].iloc[-1] > threshold_positive:
        stock_dict['flag'] = 1
        #short_listed_stocks.append({'symbol': symbol, 'flag': 1, 'percent_return': stock_data['% Return'].iloc[-1]})
    elif stock_data['% Return'].iloc[-1] < threshold_negative:
        #short_listed_stocks.append({'symbol': symbol, 'flag': -1, 'percent_return': stock_data['% Return'].iloc[-1]})
        stock_dict['flag'] = -1
    else:
        stock_dict['flag'] = 0

    # Step 2.vi: Calculate pivots
    prev_high = stock_data['High'].iloc[-1]
    prev_low = stock_data['Low'].iloc[-1]
    prev_close = stock_data['Close'].iloc[-1]
    pivot = (prev_high + prev_low + prev_close) / 3
    res1 = 2 * pivot - prev_low
    res2 = pivot + (res1 - (2 * pivot - prev_high))
    res3 = prev_high + 2 * (pivot - prev_low)
    sup1 = 2 * pivot - prev_high
    sup2 = pivot - (res1 - sup1)
    sup3 = prev_low - 2 * (prev_high - pivot)

    # short_listed_stocks[symbol]['PIVOT'] = pivot
    # short_listed_stocks[symbol]['RES1'] = res1
    # short_listed_stocks[symbol]['RES2'] = res2
    # short_listed_stocks[symbol]['RES3'] = res3
    # short_listed_stocks[symbol]['SUP1'] = sup1
    # short_listed_stocks[symbol]['SUP2'] = sup2
    # short_listed_stocks[symbol]['SUP3'] = sup3
    stock_dict['PIVOT'] = pivot
    stock_dict['RES1'] = res1
    stock_dict['RES2'] = res2
    stock_dict['RES3'] = res3
    stock_dict['SUP1'] = sup1
    stock_dict['SUP2'] = sup2
    stock_dict['SUP3'] = sup3

    # pivot_list = [{'PIVOT': pivot}, {'RES1': res1}, {'RES2': res2}, {'RES3': res3},
    #                 {'SUP1': sup1}, {'SUP2': sup2}, {'SUP3': sup3}]
    # stock_dict.update(pivot_list)
    short_listed_stocks.append(stock_dict)

    time.sleep(0.5)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [65]:
pd.DataFrame(short_listed_stocks)

,symbol,percent_return,flag,PIVOT,RES1,RES2,RES3,SUP1,SUP2,SUP3
0,NYKAA,7.609987,1,132.750000,139.800003,143.850006,150.900009,128.699997,121.649994,117.599991
1,ZOMATO,4.709352,1,71.283333,73.066666,74.983330,76.766663,69.366669,67.583336,65.666672
2,ZYDUSLIFE,0.895983,0,517.416667,521.433309,524.866618,528.883260,513.983358,509.966715,506.533407


In [66]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os
import yaml
import time
import logging


class StockAnalyzer:
    def __init__(self):
        # Configure logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.basicConfig(level=logging.ERROR,
                            format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                            handlers=[logging.StreamHandler()])
        self.logger = logging.getLogger()
        self.logger.showLineNumber = True

        # Disable logs from yfinance module
        logging.getLogger("yfinance").setLevel(logging.WARNING)

        # Step 1: Read the stock symbols from universe.csv
        root_dir = os.path.dirname(os.getcwd())
        # Construct the absolute path to the universe.csv file
        csv_path = os.path.join(root_dir, 'data', 'universe.csv')
        # Read the universe.csv file
        self.universe_df = pd.read_csv(csv_path)
        self.stock_symbols = self.universe_df['Symbol'].tolist()

        # Read the threshold values from the YAML file
        yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
        with open(yaml_path, 'r') as yaml_file:
            config = yaml.safe_load(yaml_file)
        self.threshold_positive = config['threshold_positive']
        self.threshold_negative = config['threshold_negative']

        self.short_listed_stocks = []

    def run_analysis(self):
        for symbol in self.stock_symbols:
            stock_data = self.fetch_stock_data(symbol)
            if stock_data is None:
                continue

            percent_return = self.calculate_percent_return(stock_data)
            flag = self.check_flag(percent_return)
            pivot_points = self.calculate_pivot_points(stock_data)

            stock_dict = {
                'symbol': symbol,
                'flag': flag,
                'percent_return': percent_return,
                **pivot_points
            }
            self.short_listed_stocks.append(stock_dict)
            time.sleep(0.5)

    def fetch_stock_data(self, symbol):
        symbol_ns = symbol + ".NS"
        end_date = date.today() - timedelta(days=1)
        start_date = end_date - timedelta(days=7)
        stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

        if stock_data.empty:
            self.logger.warning(f"No data available for stock: {symbol}")
            return None

        return stock_data

    def calculate_percent_return(self, stock_data):
        stock_data['Prev Close'] = stock_data['Close'].shift(1)
        stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100
        return stock_data['% Return'].iloc[-1]

    def check_flag(self, percent_return):
        if percent_return > self.threshold_positive:
            return 1
        elif percent_return < self.threshold_negative:
            return -1
        else:
            return 0

    def calculate_pivot_points(self, stock_data):
        prev_high = stock_data['High'].iloc[-1]
        prev_low = stock_data['Low'].iloc[-1]
        prev_close = stock_data['Close'].iloc[-1]
        pivot = (prev_high + prev_low + prev_close) / 3
        res1 = 2 * pivot - prev_low
        res2 = pivot + (res1 - (2 * pivot - prev_high))
        res3 = prev_high + 2 * (pivot - prev_low)
        sup1 = 2 * pivot - prev_high
        sup2 = pivot - (res1 - sup1)
        sup3 = prev_low - 2 * (prev_high - pivot)

        pivot_points = {
            'PIVOT': pivot,
            'RES1': res1,
            'RES2': res2,
            'RES3': res3,
            'SUP1': sup1,
            'SUP2': sup2,
            'SUP3': sup3
        }

        return pivot_points


# Create an instance of StockAnalyzer and run the analysis
analyzer = StockAnalyzer()
analyzer.run_analysis()

# Access the short-listed stocks
short_listed_stocks = analyzer.short_listed_stocks

# Convert short_listed_stocks to a pandas DataFrame
df = pd.DataFrame(short_listed_stocks)
print(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
      symbol  flag  percent_return       PIVOT        RES1        RES2  \
0      NYKAA     1        7.609987  132.750000  139.800003  143.850006   
1     ZOMATO     1        4.709352   71.283333   73.066666   74.983330   
2  ZYDUSLIFE     0        0.895983  517.416667  521.433309  524.866618   

         RES3        SUP1        SUP2        SUP3  
0  150.900009  128.699997  121.649994  117.599991  
1   76.766663   69.366669   67.583336   65.666672  
2  528.883260  513.983358  509.966715  506.533407  


In [67]:
df

,symbol,flag,percent_return,PIVOT,RES1,RES2,RES3,SUP1,SUP2,SUP3
0,NYKAA,1,7.609987,132.750000,139.800003,143.850006,150.900009,128.699997,121.649994,117.599991
1,ZOMATO,1,4.709352,71.283333,73.066666,74.983330,76.766663,69.366669,67.583336,65.666672
2,ZYDUSLIFE,0,0.895983,517.416667,521.433309,524.866618,528.883260,513.983358,509.966715,506.533407


In [2]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta, datetime, time
from smartapi import SmartConnect
import time
import logging
import pytz
import yaml
import pyotp

In [3]:
def load_credentials():
    with open("../config/api_credentials.yaml", "r") as file:
        credentials = yaml.safe_load(file)
    api_key = credentials["api_key"]
    secret_key = credentials["secret_key"]
    client_id = credentials["client_id"]
    password = credentials["password"]
    print(api_key, secret_key, client_id, password)
    return api_key, secret_key, client_id, password
def generate_smart_connect_object(api_key, secret_key, client_id, password ):
    obj = SmartConnect(api_key=api_key)
    totp = pyotp.TOTP(s=secret_key)
    data = obj.generateSession(client_id, password, totp.now())
    # One time script to refresh NIFTY data
    refreshToken= data['data']['refreshToken']
    #fetch the feedtoken
    feedToken=obj.getfeedToken()
    #fetch User Profile
    userProfile= obj.getProfile(refreshToken)
    print(userProfile)
    return obj
api_key, secret_key, client_id, password = load_credentials()    
obj = generate_smart_connect_object(api_key, secret_key, client_id, password)

JyXQQd0X C5O2A5URXI5LRRVMT7Y7TPYJFM VADL1002 2112
{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'clientcode': 'VADL1002', 'name': 'BHAVESH GHODASARA', 'email': '', 'mobileno': '', 'exchanges': ['bse_cm', 'nse_cm', 'nse_fo', 'cde_fo', 'mcx_fo', 'ncx_fo'], 'products': ['BO', 'NRML', 'CO', 'CNC', 'MIS', 'MARGIN'], 'lastlogintime': '', 'broker': ''}}


In [4]:
print(obj)

In [5]:
ltp = obj.ltpData("NSE", f"PRESTIGE-EQ", "20302")
ltp

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': {'exchange': 'NSE',
  'tradingsymbol': 'PRESTIGE-EQ',
  'symboltoken': '20302',
  'open': 574.15,
  'high': 581.45,
  'low': 574.05,
  'close': 574.15,
  'ltp': 580.15}}

In [11]:
ltp['data']['ltp']

565.0

In [20]:
ltp = obj.ltpData("NSE", f"PRESTIGE-EQ", "20302")

TypeError: ltpData() missing 1 required positional argument: 'symboltoken'

In [23]:
obj.getCandleData({
        "exchange": "NSE",
            "symboltoken": "3045",
            "interval": "ONE_MINUTE",
            "fromdate": "2021-02-08 09:00", 
            "todate": "2021-02-08 09:16"}
)

{'exchange': 'NFO', 'symboltoken': '7', 'interval': 'ONE_DAY', 'fromdate': '2023-05-31 00:00', 'todate': '2023-06-07 23:59'}

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': [['2021-02-08T09:15:00+05:30', 398.0, 403.45, 397.3, 401.85, 2609877],
  ['2021-02-08T09:16:00+05:30', 402.15, 403.0, 401.45, 401.75, 1470752]]}